<a href="https://colab.research.google.com/github/DonRoboto/CursoMOO/blob/main/Tarea2_evolutivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitstring

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import random
import bitstring as bits


In [81]:
class Individuo():
    def __init__(self, x='', y=''):
      if x!='':
        self.f_x = float(x)
        self.f_y = float(y)
      else:
        self.f_x = round(random.uniform(-3, 3),2)
        self.f_y = round(random.uniform(-3, 3),2)
      self.cromosomas()

    def cromosomas(self):
        aux_f_x = bits.BitArray(float=self.f_x, length=32)
        aux_f_y = bits.BitArray(float=self.f_y, length=32)

        self.b_x = list(aux_f_x.bin)
        self.b_y = list(aux_f_y.bin)
 

    def fittnes(self):
      r1_1 = ''.join(self.b_x)
      r1_2 = ''.join(self.b_y)

      r2_1 = bits.BitArray(bin=r1_1).float
      r2_2 = bits.BitArray(bin=r1_2).float

      #funcion de test
      resultado = r2_1**2 + r2_2**2
      return resultado
        

In [109]:
class Poblacion():
  p = []

  def __init__(self, n=''):
    self.p = []
    if n!='':
      for k in range(n):
        i=Individuo()
        self.p.append(i)


In [113]:
pob_i = Poblacion(n=6)

for i in pob_i.p:
  print(i.fittnes())



8.91199954319001
8.14579957170487
3.3784999361038217
6.707599875831605
3.230800017547608
7.892200311708454


In [112]:
pob_n = Poblacion()

for i in pob_n.p:
  print(i.fittnes())

In [69]:
miV1=Individuo()

cro1_1 = miV1.b_x
cro1_2 = miV1.b_y

print(cro1_1)
print(cro1_2)
print("****")

res = miV1.fittnes()
print(res)


['1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0', '0', '0', '1']
['1', '0', '1', '1', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '0']
****
3.5425001549720783


In [70]:
x0 = round(random.uniform(-3, 3),2)
x1 = round(random.uniform(-3, 3),2)

print(x0)
print(x1)
print("****")

miV2=Individuo(x=x0, y=x1)

cro2_1 = miV2.b_x
cro2_2 = miV2.b_y

print(cro2_1)
print(cro2_1)
print("****")

res = miV2.fittnes()
print(res)


2.88
2.22
****
['0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0']
['0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1', '1', '0', '0']
****
13.22280078620912


In [45]:
#cruza
cruza1_x_top = cro1_1[:16]
cruza1_x_bot = cro1_1[16:]

cruza1_y_top = cro1_2[:16]
cruza1_y_bot = cro1_2[16:]

cruza2_x_top = cro2_1[:16]
cruza2_x_bot = cro2_1[16:]

cruza2_y_top = cro2_2[:16]
cruza2_y_bot = cro2_2[16:]

hijo_x = cruza1_x_top + cruza2_x_bot
hijo_y = cruza1_y_top + cruza2_y_bot

ch_x = hijo_x
ch_y = hijo_y

chs_x = ''.join(ch_x)
chs_y = ''.join(ch_y)

r2_1 = bits.BitArray(bin=chs_x)
r2_2 = bits.BitArray(bin=chs_y)
      
r3_1 = r2_1.float
r3_2 = r2_2.float

print(r3_1)
print(r3_2)

hijo = Individuo(r3_1, r3_2)
fit = hijo.fittnes()

print(fit)

-1.449062466621399
-0.17078125476837158
-1.6198437213897705
